In [23]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
from solana.rpc.commitment import Commitment
import time


client = Client("https://api.devnet.solana.com")


program_id = PublicKey("5S6xus14nV1cr4qfzzxwhSrnuM9PNkHqZCo1sAujAppE")
# program_id_account_meta = AccountMeta(program_id, False, False)
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)
# program_id_account_meta

client.is_connected()

True

In [24]:
seed = bytes([125,215,212,54,213,98,45,123]*4)
payer_keypair = Keypair.from_seed(seed)
client.request_airdrop(payer_keypair.public_key, 5000000000)

mint_account = Keypair.generate()

seed1 = bytes([225,215,211,54,213,18,25,122]*4)
vault_keypair = Keypair.from_seed(seed1)

payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
vault_account_meta = AccountMeta(PublicKey("G473EkeR5gowVn8CRwTSDop3zPwaNixwp62qi7nyVf4z"), False, True)

rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)

In [25]:
mint_account_meta = AccountMeta(mint_account.public_key, True, True)
associated_token_address = instructions.get_associated_token_address(payer_keypair.public_key, mint_account.public_key)
associated_account_meta = AccountMeta(associated_token_address, False, True)

(mint_authority, nonce) = PublicKey.find_program_address([b'cyrial_pda'], program_id)
(avatar_data_pda, avatar_pda_nonce) = PublicKey.find_program_address([b'avatar_data_pda', bytes(mint_account_meta.pubkey), bytes(associated_account_meta.pubkey)], program_id)
(sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda', bytes(program_id)], program_id)
metadata_program_pubkey = PublicKey("metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s")
(metadata_pda, metadata_nonce) = PublicKey.find_program_address([b'metadata', bytes(metadata_program_pubkey), bytes(mint_account_meta.pubkey)], metadata_program_pubkey)


avatar_data_pda_meta = AccountMeta(avatar_data_pda, False, True)
sysvar_clock_meta = AccountMeta(solana.sysvar.SYSVAR_CLOCK_PUBKEY, False, False)
sales_pda_meta = AccountMeta(sales_pda, False, True)
mint_authority_meta = AccountMeta(mint_authority, False, False)

accounts = [
    # program_id_account_meta,
    payer_account_meta, 
    vault_account_meta, 
    mint_account_meta, 
    rent_account_meta, 
    associated_account_meta,
    spl_program_meta,
    AccountMeta(SYS_PROGRAM_ID, False, False),
    mint_authority_meta,
    avatar_data_pda_meta,
    sales_pda_meta,
    AccountMeta(metadata_pda, False, True),
    sysvar_clock_meta,
]
temp_cnt = 0

temp_mints = ["FBcpzackX3aSNUhaKZQGi67EjvWyD4doFgY8CHo8fYtr", "2wiMp2EVAZMmiaNzt3C7aYsx2GEQPNspPYQDg1yymSoy", "EPwhwugsyJyRpVUGn3Npt5FZmkR1Z74doLrmgMUqtSnN"]

for mint in temp_mints:
    mint = PublicKey(mint)
    temp_associated = instructions.get_associated_token_address(payer_keypair.public_key, mint)
    temp_pda, _ = PublicKey.find_program_address([b"avatar_data_pda",
            bytes(mint),
            bytes(temp_associated)], program_id)
    print(temp_pda)
    accounts.append(AccountMeta(temp_associated, False, True))
    accounts.append(AccountMeta(mint, False, True))
    accounts.append(AccountMeta(temp_pda, False, True))
    temp_cnt += 1



while temp_cnt > 0 :
    accounts.append(spl_program_meta)
    temp_cnt -= 1

accounts = accounts + [spl_program_meta,
    AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
    spl_program_meta,
    spl_program_meta,
    AccountMeta(metadata_program_pubkey, False, False),
    ]


GXhgqCsDeAiDCpMr3MrYEJ2uYHGKVepALEpUYWWxNbEH
BgqCxeXcJRJREVwBg8DTnnJAUWmhAZcHwrbk4qNqa8wL
BvggzwsB7fm3D5DDEJ8d8i4DkpDBcEWtTkzB3DCxgKDR


In [26]:
transaction = Transaction()
transaction.add(TransactionInstruction(
    accounts,
    program_id,
    bytes([4,1])
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account)['result']}?cluster=devnet"

'https://explorer.solana.com/tx/3QXfvgeVm97NondNqTfwEZWjHsFk8soFUfZsFWEbY1onasMyBPAr8EWHdENjQoDccTYvnL7rXaCf7XzdioVcU12?cluster=devnet'